# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 59 new papers today
          27 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/26 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.09752


extracting tarball to tmp_2212.09752...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.09763


extracting tarball to tmp_2212.09763...

...

extracting tarball to tmp_2212.10512...

 done.


Retrieving document from  https://arxiv.org/e-print/2212.10517


extracting tarball to tmp_2212.10517...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.09766-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.09766) | **Comparing the Locations of Supernovae to CO (2-1) Emission in their Host  Galaxies**  |
|| Ness Mayker Chen, et al. -- incl., <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Frank Bigiel</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-12-21*|
|*Comments*| *Accepted to ApJ; 38 pages, 12 figures, 7 tables in two-column AASTEX63 format*|
|**Abstract**| We measure the molecular gas environment near recent ($< 100$ yr old) supernovae (SNe) using $\sim1''$ or $\leq 150$pc resolution CO (2-1) maps from the PHANGS-ALMA survey of nearby star-forming galaxies. This is arguably the first such study to approach the scales of individual massive molecular clouds ($M_{\rm mol} \gtrsim 10^{5.3}$ M$_{\odot}$). Using the Open Supernova Catalog (OSC), we identify 63 SNe within the PHANGS-ALMA footprint. We detect CO (2-1) emission near $\sim60\%$ of the sample at 150pc resolution, compared to $\sim35\%$ of map pixels with CO (2-1) emission, and up to $\sim95\%$ of the SNe at 1kpc resolution compared to $\sim80\%$ of map pixels with CO (2-1) emission. We expect the $\sim60\%$ of SNe within the same 150pc beam as a GMC will likely interact with these clouds in the future, consistent with the observation of widespread SN-molecular gas interaction in the Milky Way, while the other $\sim40\%$ of SNe without strong CO (2-1) detections will deposit their energy in the diffuse interstellar medium (ISM), perhaps helping drive large-scale turbulence or galactic outflows. Broken down by type, we detect CO (2-1) emission at the sites of $\sim85\%$ of our 9 stripped-envelope SNe (SESNe), $\sim40\%$ of our 34 Type II SNe, and $\sim35\%$ of our 13 Type Ia SNe, indicating that SESNe are most closely associated with the brightest CO (2-1) emitting regions in our sample. Our results confirm that SN explosions are not restricted to only the densest gas, and instead exert feedback across a wide range of molecular gas densities. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.10512-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.10512) | **PHANGS-JWST First Results: Measuring PAH Properties across the  multiphase ISM**  |
|| Jérémy Chastenet, et al. -- incl., <mark>Daizhong Liu</mark>, <mark>Frank Bigiel</mark>, <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-12-21*|
|*Comments*| *12 pages, 6 figures. Accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ*|
|**Abstract**| Ratios of polycyclic aromatic hydrocarbon (PAH) vibrational bands are a promising tool for measuring the properties of the PAH population and their effect on star formation. The photometric bands of the MIRI and NIRCam instruments on JWST provide the opportunity to measure PAH emission features across entire galaxy disks at unprecedented resolution and sensitivity. Here we present the first results of this analysis in a sample of three nearby galaxies: NGC 628, NGC 1365, and NGC 7496. Based on the variations observed in the 3.3, 7.7, and 11.3 $\mu$m features, we infer changes to the average PAH size and ionization state across the different galaxy environments. High values of F335M$_{\rm PAH}$/F1130W and low values of F1130W/F770W are measured in H II regions in all three galaxies. This suggests that these regions are populated by hotter PAHs, and/or that the PAH ionization fraction is larger. We see additional evidence of heating and/or changes in PAH size in regions with higher molecular gas content as well as increased ionization in regions with higher H$\alpha$ intensity. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.09763-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.09763) | **The Late Time Optical Evolution of Twelve Core-Collapse Supernovae:  Detection of Normal Stellar Winds**  |
|| <mark>M. Rizzo Smith</mark>, C. S. Kochanek, J. M. M. Neustadt |
|*Appeared on*| *2022-12-21*|
|*Comments*| *22 pages, 14 figures, 6 tables, submitting to MNRAS*|
|**Abstract**| We analyze the late time evolution of 12 supernovae (SNe) occurring over the last ${\sim}$41 years, including nine Type IIP/L, two IIb, and one Ib/c, using UBVR optical data from the Large Binocular Telescope (LBT) and difference imaging. We see late time (5 to 42 years) emission from nine of the eleven Type II SNe (eight Type IIP/L, one IIb). We consider radioactive decay, circumstellar medium (CSM) interactions, pulsar/engine driven emission, dust echoes, and shock perturbed binary companions as possible sources of emission. The observed emission is most naturally explained as CSM interactions with the normal stellar winds of red supergiants with mass loss rates in the range $-7.9\lesssim \log_{10}(M_\odot\text{yr}{}^{-1}) \lesssim-4.8$. We also place constraints on the presence of any shock heated binary companion to the Type Ib/c SN 2012fh and provide progenitor photometry for the Type IIb SN 2011dh, the only one of the six SNe with pre-explosion LBT observations where the SN has faded sufficiently to allow the measurement. The results are consistent with measurements from pre-explosion Hubble Space Telescope (HST) images. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.10133-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.10133) | **Search and characterization of remnant radio galaxies in the XMM$-$LSS  deep field**  |
|| Sushant Dutta, et al. -- incl., <mark>Ian Heywood</mark> |
|*Appeared on*| *2022-12-21*|
|*Comments*| *23 pages, 27 figures, 6 tables. Accepted for publication in Astrophysical Journal*|
|**Abstract**| The remnant phase of a radio galaxy is characterized by the cessation of AGN activity resulting in the stoppage of jets supplying plasma to radio lobes. In this paper, we present the search and characterization of remnant candidates in 12.5 deg$^{2}$ of the {\em XMM$-$Newton} Large$-$Scale Structure (XMM$-$LSS) field by using deep radio observations at 325 MHz from the Giant Metrewave Radio Telescope (GMRT), at 150 MHz from the LOw Frequency ARray (LOFAR), at 1.4 GHz from the Jansky Very Large Array (JVLA), and at 3 GHz from the VLA Sky Survey (VLASS). By using both morphological criteria {\viz}undetected radio core as well as spectral criteria {\viz}high spectral curvature, and ultra$-$steep spectrum, we identify 21 remnant candidates that are found to reside mostly in non$-$cluster environments, and exhibit diverse properties in terms of morphology, spectral index ($\alpha_{\rm 150}^{\rm 1400}$ in the range of $-1.71$ to $-0.75$ with a median of $-1.10$), and linear radio size (ranging from 242 kpc to 1.3 Mpc with a median of 469 kpc). Our study attempts to identify remnant candidates down to the flux density limit of 6.0 mJy at 325 MHz, and yields an upper limit on the remnant fraction ($f_{\rm rem}$) to be around 5$\%$. The observed $f_{\rm rem}$ seems consistent with the predictions of an evolutionary model assuming power law distributions of the duration of active phase and jet kinetic power with index $-0.8$ to $-1.2$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.10178-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.10178) | **Impact of Asymmetrical Mass Ejection from Proto-White Dwarfs on the  Properties of Binary Millisecond Pulsars**  |
|| Wen-Shi Tang, Shi-Jie Gao, <mark>Xiang-Dong Li</mark> |
|*Appeared on*| *2022-12-21*|
|*Comments*| *11pages, 7 figures, 2 tables; accepted by Monthly Notices of the Royal Astronomical Society (MNRAS)*|
|**Abstract**| The standard formation theory of binary millisecond pulsars (BMSPs) predicts efficient orbital circularization due to tidal interaction during the previous mass transfer phase. Therefore, BMSPs are expected to have a circular orbit. However, the discovery of several eccentric BMSPs (eBMSPs) with a white dwarf (WD) companion has challenged this picture. In particular, recent observation reveals that the spin angular momentum of the eBMSP J0955$-$6150 is tilted at an angle $>4.8^{\rm \degree}$ from the orbital angular momentum. This is the first time that a tilt angle is deduced for eBMSPs, which provides an important clue to their formation mechanism. Both the orbital eccentricity and tilt angle could be qualitatively accounted for by asymmetrical mass ejection during thermonuclear flashes from proto-WDs (so-called the thermonuclear rocket model), but detailed studies are still lacking. In this paper, we simulate the impact of the kick caused by asymmetrical mass ejection on the properties of BMSPs. We find that the thermonuclear rocket model can potentially explain the observational characteristics of both eBMSPs and normal BMSPs under reasonable input parameters. In addition, our results predict a wide range of the orbital period (from less than one day to more than several hundred days) for eBMSPs, which can be tested by future observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.10232-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.10232) | **The halo concentration and mass relation traced by satellite galaxies**  |
|| Qing Gu, et al. -- incl., <mark>Qi Guo</mark>, <mark>Tianchi Zhang</mark>, <mark>Quan Guo</mark>, <mark>Liang Gao</mark> |
|*Appeared on*| *2022-12-21*|
|*Comments*| *8 pages, 4 figures, accepted for publication in RAA*|
|**Abstract**| We study the relation between halo concentration and mass (c-M relation) using the Seventh and Eighth Data Release of the Sloan Digital Sky Survey (SDSS DR7 and DR8) galaxy catalogue. Assuming that the satellite galaxies follow the distribution of dark matter, we derive the halo concentration by fitting the satellite radial profile with a Nararro Frank and White (NFW) format. The derived c-M relation covers a wide halo mass range from $10^{11.6}$ to $10^{14.1} \rm\ M_\odot$. We confirm the anti-correlation between the halo mass and concentration as predicted in cosmological simulations. Our results are in good agreement with those derived using galaxy dynamics and gravitational lensing for halos of $10^{11.6}-10^{12.9} \rm\ M_\odot$, while they are slightly lower for halos of $10^{12.9}-10^{14.1}\rm\ M_\odot$. It is because blue satellite galaxies are less concentrated, especially in the inner regions. Instead of using all satellite galaxies, red satellites could be better tracers of the underlying dark matter distribution in galaxy groups. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.10372-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.10372) | **Se-ResNet+SVM model: an effective method of searching for hot subdwarfs  from LAMOST**  |
|| Cheng Zhongding, et al. -- incl., <mark>Zhang Yatao</mark>, <mark>Liu Meng</mark> |
|*Appeared on*| *2022-12-21*|
|*Comments*| **|
|**Abstract**| In this paper, we apply the feature-integration idea to fuse the abstract features extracted by Se-ResNet with experience features into hybrid features and input the hybrid features to the Support Vector Machine (SVM) to classify Hot subdwarfs. Based on this idea, we construct a Se-ResNet+SVM model, including a binary classification model and a four-class classification model. The four-class classification model can further screen the hot subdwarf candidates obtained by the binary classification model. The F1 values derived by the binary and the four-class classification model on the test set are 96.17% and 95.64%, respectively. Then, we use the binary classification model to classify 333,534 nonFGK type spectra in the low-resolution spectra of LAMOST DR8 and obtain a catalog of 3,266 hot subdwarf candidates, of which 1223 are newly-determined. Subsequently, the four-class classification model further filtered the 3,266 candidates, 409 and 296 are newly-determined respectively when the thresholds were set at 0.5 and 0.9. Through manual inspection, The true number of hot subdwarfs in the three newly-determined canditates are 176, 63, and 41, the corresponding precision of the classification model in the three cases are 67.94%, 84.88%, and 87.60%, respectively. Finally, we train a Se-ResNet regression model with MAE values of 1212.65 K for Teff, 0.32 dex for log g and 0.24 for [He/H], and predict the atmospheric parameters of these 176 hot subdwarf stars. This provides a certain amount of samples to help for future studies of hot subdwarfs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.10393-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.10393) | **The Circular Velocity Curve of the Milky Way from 5 to 25 kpc using  luminous red giant branch star**  |
|| Yuan Zhou, <mark>Xinyi Li</mark>, Yang Huang, <mark>Huawei Zhang</mark> |
|*Appeared on*| *2022-12-21*|
|*Comments*| *16 pages, 13 figures and 5 tables, accepted by ApJ*|
|**Abstract**| We present a sample of 254,882 luminous red giant branch (LRGB) stars selected from the APOGEE and LAMOST surveys. By combining photometric and astrometric information from the 2MASS and Gaia surveys, the precise distances of the sample stars are determined by a supervised machine learning algorithm: the gradient boosted decision trees. To test the accuracy of the derived distances, member stars of globular clusters (GCs) and open clusters (OCs) are used. The tests by cluster member stars show a precision of about 10 per cent with negligible zero-point offsets, for the derived distances of our sample stars. The final sample covers a large volume of the Galactic disk(s) and halo of $0<R<30$ kpc and $|Z|\leqslant15$ kpc. The rotation curve (RC) of the Milky Way across radius of $5\lesssim R\lesssim25$ kpc have been accurately measured with $\sim$ 54,000 stars of the thin disk population selected from the LRGB sample. The derived RC shows a weak decline along $R$ with a gradient of $-1.83\pm0.02$ $({\rm stat.}) \pm 0.07$ $({\rm sys.})$ km s$^{-1}$ kpc$^{-1}$, in excellent agreement with the results measured by previous studies. The circular velocity at the solar position, yielded by our RC, is $234.04\pm0.08$ $({\rm stat.}) \pm 1.36$ $({\rm sys.})$ km s$^{-1}$, again in great consistent with other independent determinations. From the newly constructed RC, as well as constraints from other data, we have constructed a mass model for our Galaxy, yielding a mass of the dark matter halo of $M_{\rm{200}}$ = ($8.05\pm1.15$)$\times$10$^{11} \rm{M_\odot}$ with a corresponding radius of $R_{\rm{200}}$ = $192.37\pm9.24$ kpc and a local dark matter density of $0.39\pm0.03$ GeV cm$^{-3}$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.09766.md
    + _build/html/tmp_2212.09766/./CDF_Mass_ALLSNe.png
    + _build/html/tmp_2212.09766/./CDF_Mass_ByType.png
    + _build/html/tmp_2212.09766/./ModelCDFs55.png
exported in  _build/html/2212.10512.md
    + _build/html/tmp_2212.10512/./ratios_all_maps_ngc0628_v2.png
    + _build/html/tmp_2212.10512/./ratios_1d_hists.png
    + _build/html/tmp_2212.10512/./ratios_all_maps_ngc1365_v2.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\cotwo}{\mbox{\rm CO(2\text{--}1)}}$
$\newcommand{\halpha}{\mbox{\rm H\alpha}}$
$\newcommand{\xco}{\mbox{X_{\rm CO}}}$
$\newcommand{\xcounits}{\mbox{cm^{-2} (K~km~s^{-1})^{-1}}}$
$\newcommand{\acounits}{\mbox{M_\odot~pc^{-2} (K~km~s^{-1})^{-1}}}$
$\newcommand{\sigsfr}{\mbox{\Sigma_{\rm SFR}}}$
$\newcommand{\ANU}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\ARI}{\affil{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\ASTROThreeD}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{\Bonn}{\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}}$
$\newcommand{\Carnegie}{\affiliation{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{\CCAPP}{\affil{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{\CITA}{\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}}$
$\newcommand{\CNRS}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\COOL}{\affil{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{\ITA}{\affil{Instit\"ut f\"{u}r Theoretische Astrophysik, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, Albert-Ueberle-Strasse 2, 69120 Heidelberg, Germany}}$
$\newcommand{\McMaster}{\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{\MPIA}{\affil{Max Planck Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\NRAO}{\affil{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{\OSU}{\affil{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{\OSUPhysics}{\affil{Department of Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{\Toulouse}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\UChile}{\affiliation{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{\uwyo}{\affil{Department of Physics \& Astronomy, University of Wyoming, Laramie, WY 82071 USA}}$
$\newcommand{\thefigure}{A\arabic{figure}}$
$\newcommand{\thetable}{A\arabic{table}}$
$\newcommand{\theequation}{A\arabic{equation}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\cotwo$}{\mbox{\rm CO(2\text{--}1)}}$
$\newcommand{$\halpha$}{\mbox{\rm H\alpha}}$
$\newcommand{$\xco$}{\mbox{X_{\rm CO}}}$
$\newcommand{$\xco$units}{\mbox{cm^{-2} (K~km~s^{-1})^{-1}}}$
$\newcommand{$\acounits$}{\mbox{M_\odot~pc^{-2} (K~km~s^{-1})^{-1}}}$
$\newcommand{$\sigsfr$}{\mbox{\Sigma_{\rm SFR}}}$
$\newcommand{$\ANU$}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{$\ARI$}{\affil{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{$\ASTROThreeD$}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{$\Bonn$}{\affiliation{Argelander-Institut f\"{u}r Astronomie, Universit\"{a}t Bonn, Auf dem H\"{u}gel 71, 53121 Bonn, Germany}}$
$\newcommand{$\Carnegie$}{\affiliation{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{$\CCAPP$}{\affil{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{$\CITA$}{\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}}$
$\newcommand{$\CNRS$}{\affil{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\COOL$}{\affil{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{$\ITA$}{\affil{Instit\"ut f\"{u}r Theoretische Astrophysik, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, Albert-Ueberle-Strasse 2, 69120 Heidelberg, Germany}}$
$\newcommand{$\McMaster$}{\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{$\MPIA$}{\affil{Max Planck Institut f\"{u}r Astronomie, K\"{o}nigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{$\NRAO$}{\affil{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{$\OSU$}{\affil{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{$\OSU$Physics}{\affil{Department of Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{$\Toulouse$}{\affil{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{$\UChile$}{\affiliation{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{$\uwyo$}{\affil{Department of Physics \& Astronomy, University of Wyoming, Laramie, WY 82071 USA}}$
$\newcommand{$\thefigure$}{A\arabic{figure}}$
$\newcommand{$\thetable$}{A\arabic{table}}$
$\newcommand{$\theequation$}{A\arabic{equation}}$</div>



<div id="title">

# Comparing the Locations of Supernovae to CO (2-1) Emission in their Host Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.09766-b31b1b.svg)](https://arxiv.org/abs/2212.09766)<mark>Appeared on: 2022-12-21</mark> - _Accepted to ApJ; 38 pages, 12 figures, 7 tables in two-column AASTEX63 format_

</div>
<div id="authors">

Ness~Mayker~Chen, et al. -- incl., <mark>Annie~Hughes</mark>, <mark>Kathryn~Kreckel</mark>, <mark>Frank Bigiel</mark>, <mark>Eva Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We measure the molecular gas environment near recent ($< 100$yr old) supernovae (SNe) using$\sim1"$or$\leq 150$pc resolution CO (2-1) maps from the PHANGS-ALMA survey of nearby star-forming galaxies. This is arguably the first such study to approach the scales of individual massive molecular clouds ($M_{\rm mol} \gtrsim 10^{5.3}$M$_{\odot}$). Using the Open Supernova Catalog (OSC), we identify 63 SNe within the PHANGS--ALMA footprint. We detect CO (2-1) emission near$\sim60\%$of the sample at 150 pc resolution, compared to$\sim35\%$of map pixels with CO (2-1) emission,  and up to$\sim95\%$of the SNe at 1 kpc resolution compared to$\sim80\%$of map pixels with CO (2-1) emission. We expect the$\sim60\%$of SNe within the same 150 pc beam as a GMC will likely interact with these clouds in the future, consistent with the observation of widespread SN-molecular gas interaction in the Milky Way, while the other$\sim40\%$of SNe without strong CO (2-1) detections will deposit their energy in the diffuse interstellar medium (ISM), perhaps helping drive large-scale turbulence or galactic outflows. Broken down by type, we detect CO (2-1) emission at the sites of$\sim85\%$of our 9 stripped-envelope SNe (SESNe),$\sim40\%$of our 34 Type II SNe, and$\sim35\%$of our 13 Type Ia SNe, indicating that SESNe are most closely associated with the brightest CO (2-1) emitting regions in our sample. Our results confirm that SN explosions are not restricted to only the densest gas, and instead exert feedback across a wide range of molecular gas densities.

</div>

<div id="div_fig1">

<img src="tmp_2212.09766/./CDF_Mass_ALLSNe.png" alt="Fig9" width="100%"/>

**Figure 9. -** \textit{CDFs of physical area (top panel) and molecular gas mass (bottom panel) as a function of projected molecular gas surface density at the locations of our SN sample at 150 pc resolution.} We classify the measurements into detections with signal-to-noise $\geq$ 3 (purple circles) and non-detections plotted at the upper-limit of three times the noise value of the measurement (turquoise triangles). The top panel weights each pixel by its physical area in $pc^2$ in the plane of the galaxy, while the bottom panel weights each pixel by the molecular mass within the pixel. In the top panel we generate the CDF using both non-detections and detections. When calculating the CDF, we assume that the non-detections all have lower $\Sigma_{\rm mol}$ than the lowest detection value. The bottom panel omits non-detections, assuming the pixels to have negligible molecular gas. For comparison, we also plot the CDF of all of the pixels in all the CO (2-1) maps in our sample, classifying them by signal-to-noise with dark-grey, solid (detections) and dark-grey, dashed (upper-limits) lines. Finally, we plot the average value of the map pixels with a red, vertical line and the average value of the SN sites with a blue, vertical line. Both panels show that SNe consistently explode in regions of brighter CO (2-1) emission and molecular gas surface density compared to a pixel drawn randomly from the map. (*fig:CDF_ALLSNe*)

</div>
<div id="div_fig2">

<img src="tmp_2212.09766/./CDF_Mass_ByType.png" alt="Fig10" width="100%"/>

**Figure 10. -** \textit{CDFs of physical area (top) and molecular gas mass (bottom) at the locations of SNe as a function of molecular gas surface density, now sorted by type.} As Figure \ref{fig:CDF_ALLSNe} but now separating the SNe by type. Upper-limit values are marked with triangles, and detections are marked with circles. SNe Ia have purple markers, SNe II have orange markers and SESNe (SNe IIb, Ib, and Ic) have black markers. In the top panel, the CDFs of all the pixels in the galaxy maps are also plotted, with detections marked by a solid, dark-grey curve and non-detections (with upper-limit values) with a dashed, dark-grey curve. We plot the average surface density value of map pixels with a solid, blue, vertical line; and the average surface densities at the sites of SNe Ia, SNe II, \& SESNe (IIb/Ib/Ic) with purple-dot-dashed, black-dotted, and orange-dashed vertical lines respectively. In the bottom panel, we plot the fraction of \textit{molecular gas mass} instead of the fraction of physical area. The average surface density of map pixels with detected CO (2-1) emission is plotted with a solid, blue, vertical line and the average surface density of Type Ia, II, and IIb/Ib/Ic sites with detected CO (2-1) emission are plotted with purple-dash-dotted, orange-dashed, and black-dotted vertical lines respectively. The figure shows that SESNe and Type II SNe tend to occur in regions of higher $\Sigma_{\rm mol}$  than Type Ia SNe. We also note that $\sim35\%$ of our SNe Ia occur in regions with detectable molecular gas, while $\sim65\%$ of our SNe Ia occur with no detectable CO (2-1) emission. (*fig:CDFByType*)

</div>
<div id="div_fig3">

<img src="tmp_2212.09766/./ModelCDFs55.png" alt="Fig3" width="100%"/>

**Figure 3. -** \textit{CDFs of distances to the nearest pixel with molecular gas surface density of each galaxy's individual median $3\sigma$ $\Sigma_{\rm mol}$ or higher}. Our real SNe sample is drawn with a dark-grey line. The randomly generated SNe sample is drawn with a dotted red line, gas distribution as dashed green line, stellar as dash-dotted blue line, and the hybrid model as solid purple line. The transparent shading represents the 16th-84th percentile values from 1000 random pulls each the size of our observed SN sample from each model distribution. In the bottom panel, we separate our observed SN sample by SN type. SNe Ia are marked with a dotted-black line, SNe II with a solid line, SNe Ibc with a dashed line and unclassified SNe with a dashed-dotted line. The models are plotted with the same color scheme as the top panel. (*fig:modelCDFs*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\XX}{\textbf{Flag!}\xspace}$
$\newcommand{\coto}{^{12}{\rm CO}~(2\to1)\xspace}$
$\newcommand{\halpha}{H\alpha\xspace}$
$\newcommand{\hii}{\ion{H}{2}\xspace}$
$\newcommand{\hone}{H{\sc i}\xspace}$
$\newcommand{\ration}{F1130W/F770W\xspace}$
$\newcommand{\ratioss}{F335M_{\rm PAH}/F770W\xspace}$
$\newcommand{\ratiose}{F335M_{\rm PAH}/F1130W\xspace}$
$\newcommand{\hagasratio}{I_{\rm H{\alpha}}/\Sigma_{\rm H{\sc I}+H_2}\xspace}$
$\newcommand{\rpah}{R_{\rm PAH}\xspace}$
$\newcommand{\mjysr}{MJy~sr^{-1}\xspace}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\XX$}{\textbf{Flag!}$\xspace$}$
$\newcommand{$\coto$}{^{12}{\rm CO}~(2\to1)$\xspace$}$
$\newcommand{$\halpha$}{H\alpha$\xspace$}$
$\newcommand{$\hii$}{$\ion${H}{2}$\xspace$}$
$\newcommand{$\hone$}{H{\sc i}$\xspace$}$
$\newcommand{$\ration$}{F1130W/F770W$\xspace$}$
$\newcommand{$\ratioss$}{F335M_{\rm PAH}/F770W$\xspace$}$
$\newcommand{$\ratiose$}{F335M_{\rm PAH}/F1130W$\xspace$}$
$\newcommand{$\hagasratio$}{I_{\rm H{\alpha}}/\Sigma_{\rm H{\sc I}+H_2}$\xspace$}$
$\newcommand{$\rpah$}{R_{\rm PAH}$\xspace$}$
$\newcommand{$\mjysr$}{MJy~sr^{-1}$\xspace$}$</div>



<div id="title">

# PHANGS-JWST First Results: Measuring PAH Properties across the multiphase ISM

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.10512-b31b1b.svg)](https://arxiv.org/abs/2212.10512)<mark>Appeared on: 2022-12-21</mark> - _12 pages, 6 figures. Accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ_

</div>
<div id="authors">

J\'er\'emy Chastenet, et al. -- incl., <mark>Daizhong Liu</mark>, <mark>Annie~Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Eva Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Ratios of polycyclic aromatic hydrocarbon (PAH) vibrational bands are a promising tool for measuring the properties of the PAH population and their effect on star formation.  The photometric bands of the MIRI and NIRCam instruments on JWST provide the opportunity to measure PAH emission features across entire galaxy disks at unprecedented resolution and sensitivity.Here we present the first results of this analysis in a sample of three nearby galaxies: NGC 628, NGC 1365, and NGC 7496.  Based on the variations observed in the 3.3, 7.7, and 11.3$\mu$m features, we infer changes to the average PAH size and ionization state across the different galaxy environments.  High values of$\ratiose$and low values of$\ration$are measured in$\hii$regions in all three galaxies.This suggests that these regions are populated by hotter PAHs, and/or that the PAH ionization fraction is larger.We see additional evidence of heating and/or changes in PAH size in regions with higher molecular gas content as well as increased ionization in regions with higher$\halpha$intensity.

</div>

<div id="div_fig1">

<img src="tmp_2212.10512/./ratios_all_maps_ngc0628_v2.png" alt="Fig2" width="100%"/>

**Figure 2. -** PAH band ratios in NGC 628. The maps show all pixels passing ${\rm S/N \geq 3}$ in all bands (see Section \ref{SecNoise}).
    The F335M$_{\rm PAH}$ refers to the 3.3 $\mu$m maps developed by \citet[][]{SANDSTROM2_PHANGSJWST}.
    The contour show the $\hii$ regions from the nebula catalog by \citet[][]{GROVES_HIICAT, PHANGSMUSE2022HII}.
    (The maps were arbitrarily rotated from north-up to allow for better visualization.)
     (*FigRatioNGC628*)

</div>
<div id="div_fig2">

<img src="tmp_2212.10512/./ratios_1d_hists.png" alt="Fig5" width="100%"/>

**Figure 5. -** Running medians of the $\ratioss$(top), $\ratioss$(middle), and $\ration$(bottom), in bins of $\hagasratio$(left), fraction of molecular gas, $f_{\rm H_2}$(right), and the abundance ratio $\rpah$ = (F700W+F1130W)/F2100W. (*FigPanel*)

</div>
<div id="div_fig3">

<img src="tmp_2212.10512/./ratios_all_maps_ngc1365_v2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Same as Figure \ref{FigRatioNGC628} for NGC 1365. The central pixels are masked due to saturation, using the PSF at F1130W. The visible stripes are due to $1/f$ noise in the NIRCam bands.
    (The maps were arbitrarily rotated from north-up to allow for better visualization.) (*FigRatioNGC1365*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

143  publications in the last 7 days.
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers